
# 🚗 Dynamic Pricing for Urban Parking Lots (Capstone Project)

**Summer Analytics 2025 — Consulting & Analytics Club × Pathway**

This notebook implements:
- ✅ Model 1: Baseline Linear Pricing
- ✅ Model 2: Demand-Based Pricing
- ✅ Model 3: Competitive Pricing (optional)
- ✅ Real-Time Simulation using Bokeh (batch-based streaming)


In [ ]:

import pandas as pd
import numpy as np

# Load the dataset (upload `dataset.csv` before running)
df = pd.read_csv("dataset.csv")

# Safe timestamp construction
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'].astype(str).str.strip() + ' ' + df['LastUpdatedTime'].astype(str).str.strip(),
    errors='coerce'
)
df = df.dropna(subset=['Timestamp']).copy()
df.sort_values('Timestamp', inplace=True)


In [ ]:

# Select a lot to simulate
lot_id = 'BHMBCCMKT01'
lot_df = df[df['SystemCodeNumber'] == lot_id].copy().reset_index(drop=True)

# Encode relevant fields
traffic_map = {'low': 0, 'medium': 1, 'high': 2}
vehicle_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
lot_df['TrafficLevel'] = lot_df['TrafficConditionNearby'].map(traffic_map)
lot_df['VehicleTypeWeight'] = lot_df['VehicleType'].map(vehicle_map)


In [ ]:

# Model 1: Baseline pricing
def baseline_pricing(df, base_price=10, alpha=1.5):
    prices = [base_price]
    for i in range(1, len(df)):
        prev_price = prices[-1]
        occ = df.iloc[i]['Occupancy']
        cap = df.iloc[i]['Capacity']
        prices.append(prev_price + alpha * (occ / cap))
    return prices

lot_df['BaselinePrice'] = baseline_pricing(lot_df)


In [ ]:

# Model 2: Demand-based pricing
def demand_price(row, base=10, λ=0.5, a=1, b=0.8, c=0.6, d=1.2, e=0.5):
    occ = row['Occupancy'] / row['Capacity']
    q = row['QueueLength']
    traffic = row['TrafficLevel']
    spec = row['IsSpecialDay']
    veh = row['VehicleTypeWeight']
    demand = a*occ + b*q - c*traffic + d*spec + e*veh
    norm = (demand - 5) / 10  # basic normalization
    price = base * (1 + λ * norm)
    return np.clip(price, 0.5*base, 2.0*base)

lot_df['DemandPrice'] = lot_df.apply(demand_price, axis=1)


In [ ]:

from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

def competitive_pricing(df, base_price=10, radius_km=1.0):
    prices = []
    latest = df.groupby('SystemCodeNumber').last().reset_index()
    for i, row in df.iterrows():
        comps = []
        for _, r in latest.iterrows():
            if r['SystemCodeNumber'] == row['SystemCodeNumber']:
                continue
            dist = haversine(row['Latitude'], row['Longitude'], r['Latitude'], r['Longitude'])
            if dist <= radius_km:
                comps.append(r['DemandPrice'])
        avg_comp = np.mean(comps) if comps else base_price
        own = row['DemandPrice']
        if row['Occupancy'] > 0.9 * row['Capacity'] and own > avg_comp:
            new_price = avg_comp * 0.95
        elif own < avg_comp:
            new_price = own * 1.1
        else:
            new_price = own
        prices.append(np.clip(new_price, 0.5*base_price, 2.0*base_price))
    return prices

lot_df['CompetitivePrice'] = competitive_pricing(lot_df)


In [ ]:

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from time import sleep
from IPython.display import clear_output

output_notebook()

source = ColumnDataSource(data=dict(x=[], y=[]))
p = figure(x_axis_type='datetime', title='📉 Real-Time Demand-Based Pricing', width=750, height=400)
p.line('x', 'y', source=source, line_width=2, color='green')
p.xaxis.axis_label = 'Timestamp'
p.yaxis.axis_label = 'Price ($)'
show(p)

# Simulate real-time updates
for i in range(0, len(lot_df), 8):
    batch = lot_df.iloc[i:i+8]
    new_data = dict(x=batch['Timestamp'], y=batch['DemandPrice'])
    source.stream(new_data)
    sleep(1)
